In [1]:
import os

In [2]:
os.environ["JAVA_HOME"] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ["PYSPARK_SUBMIT_ARGS"] = 'pyspark-shell'

In [3]:
os.environ["SPARK_HOME"] = '/home/rustem/Programs/spark-2.2.0-bin-hadoop2.7/'

In [4]:
import sys

sys.path.append(os.environ['SPARK_HOME']+"/python/")

In [5]:
sys.path.append(os.environ['SPARK_HOME']+"/python/lib/py4j-0.10.4-src.zip")

In [6]:
import py4j
from pyspark import SparkContext, SparkConf, SQLContext

In [7]:
conf = (SparkConf().setMaster("local[8]")
        .setAppName("ML demo")
        .set("spark.executor.memory", "2g"))

In [8]:
sc = SparkContext(conf=conf)

In [9]:
sqlcontext = SQLContext(sc)

In [10]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

In [11]:
df = sqlcontext.read.format(
    'com.databricks.spark.csv').options(
    header='true').load('train.csv')

In [12]:
df.head(3)

[Row(PassengerId=u'1', Survived=u'0', Pclass=u'3', Name=u'Braund, Mr. Owen Harris', Sex=u'male', Age=u'22', SibSp=u'1', Parch=u'0', Ticket=u'A/5 21171', Fare=u'7.25', Cabin=None, Embarked=u'S'),
 Row(PassengerId=u'2', Survived=u'1', Pclass=u'1', Name=u'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex=u'female', Age=u'38', SibSp=u'1', Parch=u'0', Ticket=u'PC 17599', Fare=u'71.2833', Cabin=u'C85', Embarked=u'C'),
 Row(PassengerId=u'3', Survived=u'1', Pclass=u'3', Name=u'Heikkinen, Miss. Laina', Sex=u'female', Age=u'26', SibSp=u'0', Parch=u'0', Ticket=u'STON/O2. 3101282', Fare=u'7.925', Cabin=None, Embarked=u'S')]

In [13]:
# добавить 5 новых фичей
# 3 фичи высчитываются из имеющихся
# хотя бы одна использует udf

# попробовать 3 новых модели

# f1 меру

# Homework
## Add new features
### Old features:

In [14]:
df = sqlcontext.read.format(
    'com.databricks.spark.csv').options(
    header='true').load('train.csv')

In [15]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import udf
from pyspark.sql import types

def parse_age(str_age):
    try:
        return float(str_age)
    except:
        return -1
    
def Embarked_transform(x):
    if x != None:
        return x
    else:
        return ''

my_udf = udf(Embarked_transform, types.StringType())
df = df.withColumn('Embarked', my_udf(df['Embarked']))

stringIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")
df = encoder.transform(indexed)
df.select("EmbarkedVec").distinct().collect()

[Row(EmbarkedVec=SparseVector(3, {2: 1.0})),
 Row(EmbarkedVec=SparseVector(3, {1: 1.0})),
 Row(EmbarkedVec=SparseVector(3, {0: 1.0})),
 Row(EmbarkedVec=SparseVector(3, {}))]

### 1) Feauture Miss/Mrs./Mr.

In [16]:
def Name_transform(x):
    if x == None:
        return 0
    elif x.find(' Mr. ') != -1:
        return 1
    elif x.find(' Mrs. ') != -1:
        return 2
    elif x.find(' Miss. ') != -1:
        return 3
    elif x.find(' Ms. ') != -1:
        return 4
    else:
        return 5
    
        


my_udf = udf(Name_transform, types.IntegerType())
df = df.withColumn('NamePrefixIndex', my_udf(df['Name']))
df.take(1)
df.select('NamePrefixIndex').distinct().collect()
encoder = OneHotEncoder(inputCol='NamePrefixIndex', outputCol='NamePrefixVec')
df = encoder.transform(df)
df.select("NamePrefixIndex").distinct().collect()

[Row(NamePrefixIndex=1),
 Row(NamePrefixIndex=3),
 Row(NamePrefixIndex=5),
 Row(NamePrefixIndex=4),
 Row(NamePrefixIndex=2)]

### 2) Ticket number

In [17]:
def Ticket_transform(x):
    if x == None:
        return -1
    else:
        try:
            z = x[x.rfind(' ')+1:]
            return int(z)
        except:
            return -1
        
my_udf = udf(Ticket_transform, types.IntegerType())
df = df.withColumn('TicketNumber', my_udf(df['Ticket']))
#df.select('TicketNumber','Ticket').distinct().collect()

### 3) Cabin letter

In [18]:
def Cabin_transform(x):
    if x == None:
        return ' '
    else:
        return x[0]
    
        

my_udf = udf(Cabin_transform, types.StringType())
df = df.withColumn('CabinLetter', my_udf(df['Cabin']))
stringIndexer = StringIndexer(inputCol="CabinLetter", outputCol="CabinLetterIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(inputCol='CabinLetterIndex', outputCol='CabinLetterVec')
df = encoder.transform(indexed)
df.select("CabinLetterVec").distinct().collect()

[Row(CabinLetterVec=SparseVector(8, {4: 1.0})),
 Row(CabinLetterVec=SparseVector(8, {3: 1.0})),
 Row(CabinLetterVec=SparseVector(8, {1: 1.0})),
 Row(CabinLetterVec=SparseVector(8, {6: 1.0})),
 Row(CabinLetterVec=SparseVector(8, {5: 1.0})),
 Row(CabinLetterVec=SparseVector(8, {2: 1.0})),
 Row(CabinLetterVec=SparseVector(8, {0: 1.0})),
 Row(CabinLetterVec=SparseVector(8, {})),
 Row(CabinLetterVec=SparseVector(8, {7: 1.0}))]

### 4) Cabin number

In [19]:
def CabinNum_transform(x):
    try:
        return int(x[2+x.rfind(' '):]) # because typical format is something like: 'A12' or 'B12 B14 B16'
    except:
        return -1
        

my_udf = udf(CabinNum_transform, types.StringType())
df = df.withColumn('CabinNum', my_udf(df['Cabin']))
#df.select("CabinNum","Cabin").distinct().collect()
df.take(2)

[Row(PassengerId=u'1', Survived=u'0', Pclass=u'3', Name=u'Braund, Mr. Owen Harris', Sex=u'male', Age=u'22', SibSp=u'1', Parch=u'0', Ticket=u'A/5 21171', Fare=u'7.25', Cabin=None, Embarked=u'S', EmbarkedIndex=0.0, EmbarkedVec=SparseVector(3, {0: 1.0}), NamePrefixIndex=1, NamePrefixVec=SparseVector(5, {1: 1.0}), TicketNumber=21171, CabinLetter=u' ', CabinLetterIndex=0.0, CabinLetterVec=SparseVector(8, {0: 1.0}), CabinNum=u'-1'),
 Row(PassengerId=u'2', Survived=u'1', Pclass=u'1', Name=u'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex=u'female', Age=u'38', SibSp=u'1', Parch=u'0', Ticket=u'PC 17599', Fare=u'71.2833', Cabin=u'C85', Embarked=u'C', EmbarkedIndex=1.0, EmbarkedVec=SparseVector(3, {1: 1.0}), NamePrefixIndex=2, NamePrefixVec=SparseVector(5, {2: 1.0}), TicketNumber=17599, CabinLetter=u'C', CabinLetterIndex=1.0, CabinLetterVec=SparseVector(8, {1: 1.0}), CabinNum=u'85')]

### 5) Length of name (One feature can be simple)

## Apply transform

In [20]:
def transf_new(r): # new transform
    return LabeledPoint(
        int(r.Survived),
        [
        #old features
            int(r.Pclass),
            r.Sex == 'male',
            float(r.Fare),
            int(r.SibSp),
            int(r.Parch),
            parse_age(r.Age),   
        ] 
        + list(r.EmbarkedVec.toArray()) +
        # new features
        [
            int(r.TicketNumber),
            int(r.CabinNum),
            len(r.Name),
        ]
        + list(r.NamePrefixVec.toArray())
        + list(r.CabinLetterVec.toArray())
    )

def transf(r): # old transform
    return LabeledPoint(
        int(r.Survived),
        [
            int(r.Pclass),
            r.Sex == 'male',
            float(r.Fare),
            int(r.SibSp),
            int(r.Parch),
            parse_age(r.Age),
        ] + list(r.EmbarkedVec.toArray())
    )

---
## Calculate f1_score

In [21]:
def acc(clf, test):
    values = test.map(lambda x: x.features)
    yhat = clf.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    errors = comp.map(lambda x: abs(x[0]-x[1]))
    return 1-errors.sum()/errors.count()

def recall(clf, test):
    values = test.map(lambda x: x.features)
    yhat = clf.predict(values)
    
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    true_pos = comp.map(lambda x: float(x[0]==x[1] and x[0] == 1.0) ).sum()
    true_cond_pos = y.sum()
    return true_pos/true_cond_pos

def precision(clf, test):
    values = test.map(lambda x: x.features)
    yhat = clf.predict(values)
    
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    true_pos = comp.map(lambda x: float(x[0]==x[1] and x[0] == 1.0) ).sum()
    pred_cond_pos = yhat.sum()
    
    return true_pos/pred_cond_pos

def f1_score(clf, test):
    #print precision(clf, test), recall(clf,test)
    return 2.0/(1.0/precision(clf, test)+1.0/recall(clf, test))


In [23]:
data = df.rdd.map(transf_new)
train, test = data.randomSplit([0.7, 0.3])

In [24]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.classification import LogisticRegressionModel,LogisticRegressionWithSGD
from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

rfc = RandomForest.trainClassifier(train, numClasses=2,
                             categoricalFeaturesInfo={},
                            numTrees=100)
lr = LogisticRegressionWithSGD.train(train)
dt = DecisionTree.trainClassifier(train, numClasses=2,categoricalFeaturesInfo={})
gb = GradientBoostedTrees.trainClassifier(train, categoricalFeaturesInfo={})

/home/rustem/Programs/spark-2.2.0-bin-hadoop2.7//python/pyspark/mllib/classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


In [25]:
print 'Random Forest       : f1 - {0} \t acc - {1}'.format(f1_score(rfc, test), acc(rfc,test))
print 'Logistic Regression : f1 - {0} \t acc - {1}'.format(f1_score(lr, test), acc(lr,test))
print 'Decision Tree       : f1 - {0} \t acc - {1}'.format(f1_score(dt, test), acc(dt,test))
print 'Gradient Boosting   : f1 - {0} \t acc - {1}'.format(f1_score(gb, test), acc(gb,test))

Random Forest       : f1 - 0.758620689655 	 acc - 0.800711743772
Logistic Regression : f1 - 0.580808080808 	 acc - 0.409252669039
Decision Tree       : f1 - 0.73786407767 	 acc - 0.807829181495
Gradient Boosting   : f1 - 0.740088105727 	 acc - 0.790035587189
